# Prompt templates
We can use Python built-in templates with substitutions, but it makes it difficult to substitute in real-time:

In [2]:
from langchain_google_vertexai import ChatVertexAI


llm = ChatVertexAI(model="gemini-2.5-flash-lite")

prompt_template = (
    "Given a job description, decide whether it suites a junior Java developer."
    "\nJOB DESCRIPTION:\n{job_description}\n"
)

job_description = """
SPS-Software Engineer (m/w/d) im Maschinenbau
Glaston Germany GmbH
Neuhausen-Hamberg
Feste Anstellung
Homeoffice möglich, Vollzeit
Erschienen: vor 1 Tag
Glaston Germany GmbH logo
SPS-Software Engineer (m/w/d) im Maschinenbau
Glaston Germany GmbH
slide number 1slide number 2slide number 3
Glaston ist eine internationale Marke mit weltweit führenden Unternehmen, die für zukunftsweisende Maschinen, Anlagen, Systeme und Dienstleistungen in der Bearbeitung von Architektur-, Fahrzeug- und Displayglas steht.

Mit unserer über 50-jährigen Erfahrung am Standort Glaston Germany GmbH in Neuhausen bei Pforzheim verbessern und sichern wir nachhaltig die Produktivität unserer Kunden bei der Fertigung von Architekturglas. Diesen Erfolg verdanken wir unseren motivierten und engagierten Mitarbeitenden und wurden so zu einem der führenden Anbieter von automatisierten und kundenspezifischen Anlagen.

Der Umgang mit Software liegt dir im Blut und du möchtest bei einem Hidden Champion durchstarten?
Dein Faible für Softwarelösungen und dein Herz für unterschiedliche Technologien sind ideale Voraussetzungen, um Maschinen wieder zu alter Stärke zu verhelfen?
Du hast einen ausgeprägten Servicegedanken und Spaß an der Arbeit mit Kunden?

Dann komm zu Glaston! Wir suchen ab sofort für unseren Bereich Service Upgrades Verstärkung!

SPS-SOFTWARE ENGINEER (M/W/D) IM MASCHINENBAU

Als SPS-Software Engineer (m/w/d) im Maschinenbau sind deine Aufgabengebiete:
Ausarbeitung und Weiterentwicklung von Kundenaufträgen und Upgrade-Konzepten
Selbstständige und termingerechte Bearbeitung von Kundenprojekten und Bereitstellung der notwendigen Dokumente
Unterstützung des Inbetriebnahme- und Servicepersonals im Haus und beim Kunden vor Ort
Diese Anforderungen musst du mitbringen:
Qualifizierte technische Ausbildung: Techniker, Studium oder vergleichbare Qualifikation
Mehrjährige Berufserfahrung im Serviceumfeld, idealerweise im Maschinen- und Anlagenbau
Umfangreiche Kenntnisse in verschiedenen SPS-Programmiersprachen (z.B. S7Classic, TIA, Simotion)
Bei uns profitierst du von folgenden Benefits:
Exzellente Rahmenbedingungen (z.B. attraktives Gehaltsmodell, flexible Arbeitszeiten mit Gleitzeit und Homeoffice-Möglichkeiten)
Attraktives Arbeitsumfeld in idyllisch-ländlicher Lage
Umfangreiche Mobilitätsförderung (z.B. Ladestation für Elektroautos)
Wellbeing am Arbeitsplatz
"""

result = llm.invoke(prompt_template.format(job_description=job_description))
print(result)

content='This job description **does not seem to be a good fit for a junior Java developer.**\n\nHere\'s why:\n\n*   **Primary Focus is on PLC Programming (SPS):** The title itself, "SPS-Software Engineer (m/w/d) im Maschinenbau," clearly indicates the core technology is SPS (Programmable Logic Controller) programming. This is a specialized field within automation and industrial control systems, vastly different from typical Java development.\n*   **Required Skills are PLC-Specific:** The "Anforderungen" (requirements) section explicitly mentions "Umfangreiche Kenntnisse in verschiedenen SPS-Programmiersprachen (z.B. S7Classic, TIA, Simotion)." These are specific to industrial automation, not Java.\n*   **Context is Machine Building:** The job is within the context of "Maschinenbau" (mechanical engineering/machine building). This means the software development will be geared towards controlling and automating machinery, not building general-purpose applications, web services, or enterp

In [3]:
from langchain_core.prompts import PromptTemplate

lc_prompt_template = PromptTemplate.from_template(prompt_template)
lc_prompt_template.invoke({"job_description": "fake_jd"})

StringPromptValue(text='Given a job description, decide whether it suites a junior Java developer.\nJOB DESCRIPTION:\nfake_jd\n')

In [4]:
from langchain_core.output_parsers import StrOutputParser

lc_prompt_template = PromptTemplate.from_template(prompt_template)
chain = lc_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Let\'s break down this job description to see if it suits a junior Java developer.\n\n**Key aspects of the job description:**\n\n*   **Title:** SPS-Software Engineer (m/w/d) im Maschinenbau (SPS Software Engineer in Mechanical Engineering)\n*   **Company:** Glaston Germany GmbH (Focus on glass processing machinery)\n*   **Core Responsibilities:**\n    *   Development and further development of customer orders and upgrade concepts.\n    *   Independent and timely processing of customer projects and provision of necessary documents.\n    *   Support of commissioning and service personnel on-site and at the customer\'s location.\n*   **Required Skills/Qualifications:**\n    *   Qualified technical training (Technician, degree, or comparable qualification).\n    *   **Several years of professional experience in the service environment, ideally in mechanical and plant engineering.**\n    *   **Extensive knowledge in various PLC programming languages (e.g., S7Classic, TIA, Simotion).**\n\n*

In [5]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import SystemMessagePromptTemplate


msg_template = HumanMessagePromptTemplate.from_template(prompt_template)
msg_example = msg_template.format(job_description="fake_jd")

chat_prompt_template = ChatPromptTemplate.from_messages(
    [SystemMessage(content="You are a helpful assistant."), msg_template]
)
chain = chat_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Let\'s break down this job description to see if it suits a junior Java developer.\n\n**Here\'s a strong NO for a junior Java developer, and why:**\n\n*   **Core Technologies:** The job description explicitly mentions "SPS-Software Engineer" and lists required skills like "S7Classic, TIA, Simotion." These are **PLC (Programmable Logic Controller) programming languages** used in industrial automation and machine control. This is a completely different domain than Java development.\n*   **Industry Focus:** The role is within **"Maschinenbau" (mechanical engineering)** and specifically deals with "Bearbeitung von Architektur-, Fahrzeug- und Displayglas" (processing of architectural, automotive, and display glass). The focus is on controlling and upgrading machinery, not general software application development.\n*   **Experience Level:** While it doesn\'t explicitly state "senior," the requirement for "Mehrjährige Berufserfahrung im Serviceumfeld, idealerweise im Maschinen- und Anlagenba

In [6]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("human", prompt_template)]
)
chain = chat_prompt_template | llm | StrOutputParser()
chain.invoke({"job_description": job_description})

'Based on the provided job description, this role is **NOT a good fit for a junior Java developer.**\n\nHere\'s why:\n\n*   **Primary Focus is SPS and Machine Engineering:** The title itself, "SPS-Software Engineer (m/w/d) im Maschinenbau," clearly indicates the focus is on **SPS (Speicherprogrammierbare Steuerung - Programmable Logic Controller)** programming and **mechanical engineering (Maschinenbau)**. Java is not mentioned as a required or even a secondary skill.\n*   **Required Technical Skills:** The description explicitly asks for:\n    *   "Umfangreiche Kenntnisse in verschiedenen SPS-Programmiersprachen (z.B. S7Classic, TIA, Simotion)" - Extensive knowledge in various SPS programming languages.\n    *   This is a completely different skillset than Java development.\n*   **Experience Level:** While not explicitly stating "senior," the requirement for "Mehrjährige Berufserfahrung im Serviceumfeld, idealerweise im Maschinen- und Anlagenbau" (Several years of professional experie